In [1]:
import numpy as np
from numba import jit
from random import randint
from time import time
from itertools import permutations



In [2]:
def create_matrix(city_count):
    city_list = np.zeros(shape=(city_count,city_count))
    for row in range(city_count):
        for column in range(city_count):
            if column==row:
                continue
            elif column < row:
                city_list[row][column] = city_list[column][row]
            else:
                city_list[row][column] = randint(1,100)
    return city_list


In [3]:
jit(nopython=True)
def permutations_fast(lista):
    if len(lista) == 0:
        return []
    if len(lista)==1:
        return [lista]
    permList = []
    for i in range(len(lista)):
        m = lista[i]
        remLista = lista[:i] + lista[i+1:]
        for p in permutations_fast(remLista):
            permList.append([m] + p)
    
    return permList


In [4]:
jit(nopython=True)
def do_sum_fast(city_matrix,path):
    sum_of_path = 0
    starting_city = path[0]
    for city in range(len(path)-1):
        sum_of_path += city_matrix[starting_city][path[city+1]]
        starting_city = path[city+1]

    sum_of_path += city_matrix[starting_city][path[0]] 
    return sum_of_path

In [5]:
jit(nopython=True)
def tsp_fast(city_numbers,city_matrix):
    min_path = 999999999
    path = []
    for perm in permutations(city_numbers):
        sum_of_perm = do_sum_fast(city_matrix,perm)
        if sum_of_perm < min_path:
            min_path = sum_of_perm
            path = perm
    return (path,min_path)


In [6]:
cities = 8
city_numbers = np.zeros(cities,dtype=np.int32)
for i in range(cities):
    city_numbers[i] = i
    
city_matrix = create_matrix(cities)
for i in city_matrix:
    print(i)

TPB = 4
threadsperblock = (TPB,TPB)
blockspergrid_x = int(np.ceil(cities)/threadsperblock[0])
blockspergrid_y = int(np.ceil(cities)/threadsperblock[1])
blockspergrid = (blockspergrid_x,blockspergrid_y)

print(blockspergrid)

print(tsp_fast(city_numbers,city_matrix))

start = time()
x = tsp_fast(city_numbers,city_matrix)
end = time()
print("Elapsed (after compilation) = %s" % (end - start))
print(x[0],x[1])


[ 0. 46. 75.  2. 16. 89. 90. 35.]
[46.  0. 60. 44. 66. 49. 35. 91.]
[75. 60.  0. 94. 67. 87. 69. 68.]
[ 2. 44. 94.  0. 49. 59. 11. 16.]
[16. 66. 67. 49.  0. 85. 19. 46.]
[89. 49. 87. 59. 85.  0. 82. 53.]
[90. 35. 69. 11. 19. 82.  0. 74.]
[35. 91. 68. 16. 46. 53. 74.  0.]
(2, 2)
((0, 3, 7, 5, 1, 2, 6, 4), 284.0)
Elapsed (after compilation) = 0.6676154136657715
(0, 3, 7, 5, 1, 2, 6, 4) 284.0
